In [9]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt

import logging
import fsspec
import fsspec.utils
import fsspec.core
import fsspec.implementations.reference as refs
import intake
import intake_xarray
import json
from tqdm import tqdm
from glob import glob
# import "fsspec-reference-maker"?
import dask
from dask.distributed import Client

In [2]:
import fsspec_reference_maker.hdf as fsshdf

In [3]:
def get_geo_data(sat,lyr,idyjl):
    # arguments
    # sat   goes-east,goes-west,himawari
    # lyr   year
    # idyjl day of year
    
    d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl)
    fs = s3fs.S3FileSystem(anon=True) #connect to s3 bucket!

    #create strings for the year and julian day
    imon,idym=d.month,d.day
    syr,sjdy,smon,sdym = str(lyr).zfill(4),str(idyjl).zfill(3),str(imon).zfill(2),str(idym).zfill(2)
    
    #use glob to list all the files in the directory
    if sat=='goes-east':
        file_location,var = fs.glob('s3://noaa-goes16/ABI-L2-SSTF/'+syr+'/'+sjdy+'/*/*.nc'),'SST'
    if sat=='goes-west':
        file_location,var = fs.glob('s3://noaa-goes17/ABI-L2-SSTF/'+syr+'/'+sjdy+'/*/*.nc'),'SST'
    if sat=='himawari':
        file_location,var = fs.glob('s3://noaa-himawari8/AHI-L2-FLDK-SST/'+syr+'/'+smon+'/'+sdym+'/*/*L2P*.nc'),'sea_surface_temperature'
    
    return file_location

In [4]:
flist = get_geo_data("goes-east", 2020, 210)
url = ["s3://" + f for f in flist]

In [7]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:36231 Dashboard: /proxy/8787/status,Cluster Workers: 4 Cores: 4 Memory: 7.52 GB


In [26]:
# This could easily be pa./jsons/ized with Dask
# @dask.delayed
def gen_json(urllist):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False, default_cache_type="none"
    )

    for u in urllist:
        with fsspec.open(u, **so) as f:
            h5chunks = fsshdf.SingleHdf5ToZarr(f, u, xarray=True)
            j = h5chunks.translate()
            fname = u.split("/")[7]
            with open(f"./jsons/{fname}.json", "w") as fout:
                fout.write(json.dumps(j))


### Test in serial

In [ ]:
%%time
gen_json(url)

### Test with Dask

In [32]:
%%time
func = dask.delayed(gen_json)(url)

CPU times: user 555 µs, sys: 0 ns, total: 555 µs
Wall time: 468 µs


#### I am not using dask correctly apparently

In [31]:
# %%time
# func.compute()